In [0]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,CuDNNLSTM,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten
from keras.layers.embeddings import Embedding
import numpy as np

## Import File from local drive to Colab Notebook

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Signals (Input Data)


In [0]:
import io
import os

def load_data(incoming):
  path="drive/My Drive/Data/Human_activity_recognition" +'/'+ incoming
  print(path)
  files=os.listdir(path)
  print(files)
  signals_data = []

  for file in files:
    filename=path+'/'+file
    df=pd.read_csv(filename, delim_whitespace=True, header=None)
    df=(df-df.mean())/(df.max()-df.min())
    signals_data.append(df.as_matrix())
    
  # Transpose is used to change the dimensionality of the output,
  # aggregating the signals by combination of sample/timestep.
  # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
  return np.transpose(signals_data, (1, 2, 0)) 

In [0]:
X_train = load_data('Train')
X_test = load_data('Test')
print(X_train.shape)
print(X_test.shape)

drive/My Drive/Data/Human_activity_recognition/Train
['body_acc_x_train.txt', 'body_acc_y_train.txt', 'body_acc_z_train.txt', 'body_gyro_x_train.txt', 'body_gyro_y_train.txt', 'body_gyro_z_train.txt', 'total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt']
drive/My Drive/Data/Human_activity_recognition/Test
['body_acc_x_test.txt', 'body_acc_y_test.txt', 'body_acc_z_test.txt', 'body_gyro_x_test.txt', 'body_gyro_y_test.txt', 'body_gyro_z_test.txt', 'total_acc_x_test.txt', 'total_acc_y_test.txt', 'total_acc_z_test.txt']
(7352, 128, 9)
(2947, 128, 9)


In [0]:
def load_op(incoming):
  """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
  path="drive/My Drive/Data/Human_activity_recognition" +'/'+ incoming +'.txt'
  
  df=pd.read_csv(path, delim_whitespace=True, header=None)[0]
  
  return pd

In [0]:
y_train=load_op('y_train')
y_test=load_op('y_test')


## Loading the Output labels bt spliting into Static and Dynamic 


1.  walking, up, down -- dynamic
2.   sitting standing lying -- static 



In [0]:
def load_op_2(incoming):
  """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
  path="drive/My Drive/Data/Human_activity_recognition" +'/'+ incoming +'.txt'
  
  df=pd.read_csv(path, delim_whitespace=True, header=None)[0]
  df[df<=3] = 0
  df[df>3] = 1
  return pd.get_dummies(df).as_matrix()

In [0]:
y_train_2=load_op_2('y_train')
y_test_2=load_op_2('y_test')
print(y_train_2.shape)
print(y_test_2.shape)

(7352, 2)
(2947, 2)


## Model for classifying data into Static and Dynamic activities

In [0]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

W0630 12:30:40.393746 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 12:30:40.411869 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 12:30:40.415036 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0630 12:30:40.459711 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0630 12:30:40.471667 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout_1 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1984)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                99250     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
Total para

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train_2, epochs=10, batch_size=16,validation_data=(X_test, y_test_2))

W0630 12:30:40.560046 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0630 12:30:40.594328 140271284651904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0630 12:30:40.712684 140271284651904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 5s 693us/step - loss: 0.0323 - acc: 0.9846 - val_loss: 0.0091 - val_acc: 0.9980
Epoch 2/10
7352/7352 [==============================] - 3s 446us/step - loss: 1.9257e-04 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 0.9973
Epoch 3/10
7352/7352 [==============================] - 3s 449us/step - loss: 9.1948e-05 - acc: 1.0000 - val_loss: 0.0085 - val_acc: 0.9983
Epoch 4/10
7352/7352 [==============================] - 3s 448us/step - loss: 1.1764e-04 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 0.9990
Epoch 5/10
7352/7352 [==============================] - 3s 444us/step - loss: 3.2361e-05 - acc: 1.0000 - val_loss: 0.0090 - val_acc: 0.9983
Epoch 6/10
7352/7352 [==============================] - 3s 451us/step - loss: 1.4335e-05 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9986
Epoch 7/10
7352/7352 [==============================] - 3s 446us/step - loss: 2.8019e-06 - acc: 1.0000 - val_loss: 0

## Save the 2 class classification model 

In [0]:
model.save('drive/My Drive/Data/Human_activity_recognition/model_2class.h5')

## Classificaton of Static activities

In [0]:
def load_op_stat(incoming):
  """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
  path="drive/My Drive/Data/Human_activity_recognition" +'/'+ incoming +'.txt'
  
  df=pd.read_csv(path, delim_whitespace=True, header=None)[0]
  df_subset=df>3
  df=df[df_subset]
  return pd.get_dummies(df).as_matrix(),df_subset

In [0]:
y_train_stat,x_train_size=load_op_stat('y_train')
y_test_stat,x_test_size=load_op_stat('y_test')
X_train_stat=X_train[x_train_size]
X_test_stat=X_test[x_test_size]
print(y_train_stat.shape)
print(y_test_stat.shape)
print(X_test_stat.shape)
print(X_train_stat.shape)


(4067, 3)
(1560, 3)
(1560, 128, 9)
(4067, 128, 9)


## Model for Static Activites

In [0]:
acc_score=[]
for f in [32,64]:
  for f1 in [16,32]:
    for k in [5,3]:
      for k1 in [5,3]:
        for d in [0.5,0.6]:
          for p in [2,1]:
            for s in [2,1]:

              model = Sequential()
              model.add(Conv1D(filters=f, kernel_size=k,padding='same', activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
              model.add(Conv1D(filters=f1, kernel_size=k1,padding='same', activation='relu',kernel_initializer='he_uniform'))
              model.add(Dropout(d))
              model.add(MaxPooling1D(pool_size=p,strides=s))

              model.add(Flatten())
              model.add(Dense(64, activation='relu',kernel_initializer='he_uniform'))
              model.add(BatchNormalization()) 
              model.add(Dropout(d))

              model.add(Dense(32, activation='relu',kernel_initializer='he_uniform'))
              model.add(BatchNormalization()) 
              model.add(Dropout(d))

              model.add(Dense(3, activation='softmax'))

              model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
              model.fit(X_train_stat,y_train_stat, epochs=10, batch_size=16,validation_data=(X_test_stat, y_test_stat))

              #Evaluate the model 
              score = model.evaluate(X_test_stat, y_test_stat)

              #Add the evaluation to a list 
              acc_score.append([f,f1,k,k1,d,p,s,score[1]])




In [0]:
df=pd.DataFrame(acc_score,columns=['conv_1','conv_2','kernel1','kernel2','dropout','poolsize','stride','accuracy'])

df

,conv_1,conv_2,kernel1,kernel2,dropout,poolsize,stride,accuracy
0,32,16,5,5,0.5,2,2,0.894872
1,32,16,5,5,0.5,2,1,0.900641
2,32,16,5,5,0.5,1,2,0.871795
3,32,16,5,5,0.5,1,1,0.887821
4,32,16,5,5,0.6,2,2,0.864103
5,32,16,5,5,0.6,2,1,0.885897
6,32,16,5,5,0.6,1,2,0.894231
7,32,16,5,5,0.6,1,1,0.878205
8,32,16,5,3,0.5,2,2,0.879487
9,32,16,5,3,0.5,2,1,0.860897


In [0]:
df[df['accuracy']==df['accuracy'].max()]

,conv_1,conv_2,kernel1,kernel2,dropout,poolsize,stride,accuracy
111,64,32,5,3,0.6,1,1,0.907692


In [0]:
df.sort_values('accuracy',ascending=False).head(10)

,conv_1,conv_2,kernel1,kernel2,dropout,poolsize,stride,accuracy
111,64,32,5,3,0.6,1,1,0.907692
47,32,32,5,3,0.6,1,1,0.903846
42,32,32,5,3,0.5,1,2,0.901923
106,64,32,5,3,0.5,1,2,0.901282
32,32,32,5,5,0.5,2,2,0.900641
1,32,16,5,5,0.5,2,1,0.900641
67,64,16,5,5,0.5,1,1,0.900000
103,64,32,5,5,0.6,1,1,0.899359
49,32,32,3,5,0.5,2,1,0.898718
66,64,16,5,5,0.5,1,2,0.898718


In [0]:
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=5,padding='same', activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
  model.add(Conv1D(filters=32, kernel_size=3,padding='same', activation='relu',kernel_initializer='he_uniform'))
  model.add(Dropout(0.6))
  model.add(MaxPooling1D(pool_size=1,strides=1))

  model.add(Flatten())
  model.add(Dense(64, activation='relu',kernel_initializer='he_uniform'))
  model.add(BatchNormalization()) 
  model.add(Dropout(0.6))

  model.add(Dense(32, activation='relu',kernel_initializer='he_uniform'))
  model.add(BatchNormalization()) 
  model.add(Dropout(0.6))

  model.add(Dense(3, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  model.fit(X_train_stat,y_train_stat, epochs=100, batch_size=16,validation_data=(X_test_stat, y_test_stat))


Train on 4067 samples, validate on 1560 samples
Epoch 1/100
4067/4067 [==============================] - 36s 9ms/step - loss: 0.4880 - acc: 0.8117 - val_loss: 0.3385 - val_acc: 0.8718
Epoch 2/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.3305 - acc: 0.8817 - val_loss: 0.3655 - val_acc: 0.8923
Epoch 3/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.2840 - acc: 0.8965 - val_loss: 0.3409 - val_acc: 0.8686
Epoch 4/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.2663 - acc: 0.9007 - val_loss: 0.2910 - val_acc: 0.8942
Epoch 5/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.2475 - acc: 0.9093 - val_loss: 0.2989 - val_acc: 0.8872
Epoch 6/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.2550 - acc: 0.9058 - val_loss: 0.2924 - val_acc: 0.8949
Epoch 7/100
4067/4067 [==============================] - 5s 1ms/step - loss: 0.2435 - acc: 0.9100 - val_loss: 0.3254 - val_acc: 0.8885
Epoch 

In [0]:
model.save('drive/My Drive/Data/Human_activity_recognition/model_stat_class.h5')

## For Dynamic Activities



In [0]:
def load_op_dyn(incoming):
  """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
  path="drive/My Drive/Data/Human_activity_recognition" +'/'+ incoming +'.txt'
  
  df=pd.read_csv(path, delim_whitespace=True, header=None)[0]
  df_subset=df<=3
  df=df[df_subset]
  return pd.get_dummies(df).as_matrix(),df_subset

In [0]:
y_train_dyn,x_train_size=load_op_dyn('y_train')
y_test_dyn,x_test_size=load_op_dyn('y_test')
X_train_dyn=X_train[x_train_size]
X_test_dyn=X_test[x_test_size]
print(y_train_dyn.shape)
print(y_test_dyn.shape)
print(X_test_dyn.shape)
print(X_train_dyn.shape)


(3285, 3)
(1387, 3)
(1387, 128, 9)
(3285, 128, 9)


## Model for Dynamic 

In [0]:
acc_score=[]
for f in [32,64]:
  for f1 in [16,32]:
    for k in [5,3]:
      for k1 in [5,3]:
        for d in [0.5,0.6]:
          for p in [2,1]:
            for s in [2,1]:

              model = Sequential()
              model.add(Conv1D(filters=f, kernel_size=k,padding='same', activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
              model.add(Conv1D(filters=f1, kernel_size=k1,padding='same', activation='relu',kernel_initializer='he_uniform'))
              model.add(Dropout(d))
              model.add(MaxPooling1D(pool_size=p,strides=s))

              model.add(Flatten())
              model.add(Dense(64, activation='relu',kernel_initializer='he_uniform'))
              model.add(BatchNormalization()) 
              model.add(Dropout(d))

              model.add(Dense(32, activation='relu',kernel_initializer='he_uniform'))
              model.add(BatchNormalization()) 
              model.add(Dropout(d))

              model.add(Dense(3, activation='softmax'))

              model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
              model.fit(X_train_dyn,y_train_dyn, epochs=10, batch_size=16,validation_data=(X_test_dyn, y_test_dyn),verbose=0)

              #Evaluate the model 
              score = model.evaluate(X_test_dyn, y_test_dyn)

              #Add the evaluation to a list 
              acc_score.append([f,f1,k,k1,d,p,s,score[1]])




1387/1387 [==============================] - 1s 616us/step


In [0]:
df=pd.DataFrame(acc_score,columns=['conv_1','conv_2','kernel1','kernel2','dropout','poolsize','stride','accuracy'])


,conv_1,conv_2,kernel1,kernel2,dropout,poolsize,stride,accuracy
0,32,16,5,5,0.5,2,2,0.932228
1,32,16,5,5,0.5,2,1,0.963230
2,32,16,5,5,0.5,1,2,0.956020
3,32,16,5,5,0.5,1,1,0.975487
4,32,16,5,5,0.6,2,2,0.979813
5,32,16,5,5,0.6,2,1,0.945926
6,32,16,5,5,0.6,1,2,0.964672
7,32,16,5,5,0.6,1,1,0.965393
8,32,16,5,3,0.5,2,2,0.944484
9,32,16,5,3,0.5,2,1,0.963951


In [0]:
df[df['accuracy']==df['accuracy'].max()]

,conv_1,conv_2,kernel1,kernel2,dropout,poolsize,stride,accuracy
99,64,32,5,5,0.5,1,1,0.984859


In [0]:
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=5,padding='same', activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
  model.add(Conv1D(filters=32, kernel_size=5,padding='same', activation='relu',kernel_initializer='he_uniform'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=1,strides=1))

  model.add(Flatten())
  model.add(Dense(64, activation='relu',kernel_initializer='he_uniform'))
  model.add(BatchNormalization()) 
  model.add(Dropout(0.5))

  model.add(Dense(32, activation='relu',kernel_initializer='he_uniform'))
  model.add(BatchNormalization()) 
  model.add(Dropout(0.5))

  model.add(Dense(3, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train_dyn,y_train_dyn, epochs=40, batch_size=8,validation_data=(X_test_dyn, y_test_dyn),verbose=1)

  #Evaluate the model 
  score = model.evaluate(X_test_dyn, y_test_dyn)

Train on 3285 samples, validate on 1387 samples
Epoch 1/40
3285/3285 [==============================] - 5s 1ms/step - loss: 1.2275 - acc: 0.5556 - val_loss: 0.4642 - val_acc: 0.8356
Epoch 2/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.4150 - acc: 0.8524 - val_loss: 0.2090 - val_acc: 0.9373
Epoch 3/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.3043 - acc: 0.9008 - val_loss: 0.1678 - val_acc: 0.9474
Epoch 4/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.2123 - acc: 0.9306 - val_loss: 0.5599 - val_acc: 0.8327
Epoch 5/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.1966 - acc: 0.9321 - val_loss: 0.2233 - val_acc: 0.9272
Epoch 6/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.1777 - acc: 0.9412 - val_loss: 0.2448 - val_acc: 0.9373
Epoch 7/40
3285/3285 [==============================] - 3s 1ms/step - loss: 0.1429 - acc: 0.9559 - val_loss: 0.1409 - val_acc: 0.9553
Epoch 8/40
328

In [0]:
model.save('drive/My Drive/Data/Human_activity_recognition/model_dyn_class.h5')

## Final Prediction pipeline

In [0]:

from keras.models import load_model
import pickle
model_2class = load_model('drive/My Drive/Data/Human_activity_recognition/model_2class.h5')
model_dynamic = load_model('drive/My Drive/Data/Human_activity_recognition/model_stat_class.h5')
model_static = load_model('drive/My Drive/Data/Human_activity_recognition/model_dyn_class.h5')

W0701 04:13:50.674447 140289857959808 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0701 04:13:53.888343 140289857959808 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0701 04:13:54.012605 140289857959808 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0701 04:13:54.128575 140289857959808 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


## Two class prediction

In [0]:
 ##predicting whether dynamic or static
predict_2class = model_2class.predict(X_test)
Y_pred_2class =  np.argmax(predict_2class, axis=1)
#static data filter
X_static = X_test[Y_pred_2class==1]
#dynamic data filter
X_dynamic = X_test[Y_pred_2class==0]

In [0]:
#predicting static activities
predict_static = model_static.predict(X_train_stat)
predict_static = np.argmax(predict_static,axis=1)
predict_static = predict_static + 4

In [0]:
#predicting dynamic activites
predict_dynamic = model_dynamic.predict(X_train_dyn)
predict_dynamic = np.argmax(predict_dynamic,axis=1)
predict_dynamic = predict_dynamic + 1

In [0]:
#appending final output to one list in the same sequence of input data
i,j = 0,0 
final_pred = []
for mask in Y_pred_2class:
    if mask == 1:
        final_pred.append(predict_static[i])
        i = i + 1
    else:
        final_pred.append(predict_dynamic[j])
        j = j + 1 

In [0]:
##accuracy of test
from sklearn.metrics import accuracy_score
print('Accuracy of train data',accuracy_score(y_test,final_pred))


ValueError: ignored

In [0]:
plt.figure(figsize=(8,8))
labels=['WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS','SITTING','STANDING','LAYING']
plot_confusion_matrix(cm, classes=labels, 
                      normalize=True, title='Normalized confusion matrix', cmap = plt.cm.Greens)
plt.show()

NameError: ignored

<Figure size 576x576 with 0 Axes>

In [0]:
final_pred

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
